In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from llama_cpp import Llama
from llama_cpp.llama_chat_format import Llava15ChatHandler
import pathlib
from PIL import Image
import io

# --- 1. Конфигурация для 13B модели ---
MODEL_PATH = "./models/llava-v1.6-vicuna-13b.Q4_K_M.gguf"
CLIP_MODEL_PATH = "./models/mmproj-model-f16.gguf"
IMG_DIR = "img"
TEMP_IMG_DIR = "img_resized" 
os.makedirs(TEMP_IMG_DIR, exist_ok=True)
RESULTS_FILE = "llava_local_13b_english_prompt_results.csv" # Новое имя файла

# <-- 1. ГЛАВНОЕ ИЗМЕНЕНИЕ: Промпт переведен на английский
# Этот формат модель поймет намного лучше и будет следовать ему точнее.
PROMPT_DETAILED_EN = """You are a cat breed identification expert. Analyze the cat in the image and provide your analysis in the following format:

**1. Verdict (Yes/No):** [Your 'Yes' or 'No' answer to the question: Is this a British Shorthair cat?]
**2. Key Features:** [Describe in single words, separated by commas, 3-4 key features that influenced your decision (e.g., head shape, ears shape, coat, color, build, eyes color).]
**3. Alternative Breed (if applicable):** [If the verdict is 'No', state the most likely breed. If 'Yes', leave this blank with a dash '-'.]
"""

# --- 2. Настройки (оставляем те же, они оптимальны) ---
N_GPU_LAYERS = 35
N_CTX = 4096
N_BATCH = 512
N_THREADS = os.cpu_count()
TEMPERATURE = 1 # Оставим небольшую "свободу" для качественных описаний
REPEAT_PENALTY = 1.1
MAX_TOKENS = 1000

# --- 3. Функция сжатия (без изменений) ---
def resize_and_save_image(original_path, output_path, max_size=(1024, 1024)):
    with Image.open(original_path) as img:
        img.thumbnail(max_size)
        if img.mode in ("RGBA", "P"): img = img.convert("RGB")
        img.save(output_path, format='JPEG')
    return output_path

# --- 4. Загрузка модели (без изменений) ---
llm = None
if os.path.exists(MODEL_PATH) and os.path.exists(CLIP_MODEL_PATH):
    print("Загрузка 13B модели...")
    try:
        chat_handler = Llava15ChatHandler(clip_model_path=CLIP_MODEL_PATH, verbose=False)
        llm = Llama(
            model_path=MODEL_PATH, chat_handler=chat_handler,
            n_ctx=N_CTX, n_gpu_layers=N_GPU_LAYERS, n_batch=N_BATCH, n_threads=N_THREADS, verbose=False
        )
        print("✅ 13B модель успешно загружена!")
    except Exception as e:
        print(f"❌ Ошибка загрузки моделей: {e}")
else:
    print(f"❌ Файл основной модели или CLIP не найден.")

# --- 5. Основной цикл обработки ---
if llm:
    try:
        # Можете изменить срез [:3], чтобы обработать все картинки
        image_files = sorted([f for f in os.listdir(IMG_DIR) if f.endswith(('.jpg', '.jpeg', '.png'))])
        results_data = []
        
        print(f"Начинаем обработку {len(image_files)} изображений (13B модель, английский промпт)...")
        for filename in tqdm(image_files, desc="LLaVA Local 13B (EN)"):
            original_image_path = os.path.join(IMG_DIR, filename)
            temp_image_path = os.path.join(TEMP_IMG_DIR, filename)
            try:
                resize_and_save_image(original_image_path, temp_image_path)
                image_uri = pathlib.Path(temp_image_path).resolve().as_uri()
                
                prompt_for_llm = f"USER: <image>\n{PROMPT_DETAILED_EN}\nASSISTANT:"
                
                response = llm(
                    prompt_for_llm,
                    stop=["USER:", "ASSISTANT:"],
                    max_tokens=MAX_TOKENS,
                    temperature=TEMPERATURE,
                    repeat_penalty=REPEAT_PENALTY
                )
                
                result_text = response['choices'][0]['text'].strip()
                results_data.append({"filename": filename, "llava_13b_response": result_text})

            except Exception as e:
                results_data.append({"filename": filename, "llava_13b_response": f"Error: {e}"})
        
        df_results = pd.DataFrame(results_data)
        df_results.to_csv(RESULTS_FILE, index=False, encoding='utf-8')
        print(f"\n✅ Результаты теста сохранены в '{RESULTS_FILE}'.")
        display(df_results.head())

    except FileNotFoundError:
        print(f"❌ Ошибка: Директория '{IMG_DIR}' не найдена.")
else:
    print("\n❌ Основной процесс не был запущен, так как модели не были загружены.")

Загрузка 13B модели...
✅ 13B модель успешно загружена!
Начинаем обработку 30 изображений (13B модель, английский промпт)...


LLaVA Local 13B (EN):  20%|██        | 6/30 [01:04<04:07, 10.32s/it]